In [ ]:
%matplotlib widget

In [ ]:
from ipywidgets import interactive, widgets
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from matplotlib.widgets import PolygonSelector
from matplotlib.backend_bases import MouseEvent

import ast
from camera import Camera
import pickle as pkl

from dorna_vision.draw import *
from dorna_vision.util import *
#from dorna_vision.detect import *

########################################################
########################################################

#??? method_value to run, abd
import json
import cv2 as cv
from dorna_vision.util import *
from dorna_vision.draw import *
from ai import * #???? change this to from dorna_vision.ai import *
import os
import time
import threading
import numpy as np

class Detection(object):
    """docstring for Detect"""
    def __init__(self, robot=None, camera=None, model_path=None):
        super(Detection, self).__init__()
        
        # init camera and robot and calib matrix 
        self.camera = camera
        self.robot = robot
        
        # pattern config ???
        self.config = {"poi_value": [], "color_enb": 0, "color_h": [60, 120], "color_s": [85, 170], "color_v": [85, 170], "color_inv": 0, "roi_enb": 0, "roi_value": [[353.84, 171.66], [324.84, 524.06], [828.9, 564.2], [811.06, 180.58]], "roi_inv": 0, "intensity_enb": 0, "intensity_alpha": 2.0, "intensity_beta": 50, "method_value": 0, "m_elp_pf_mode": 0, "m_elp_nfa_validation": 1, "m_elp_min_path_length": 50, "m_elp_min_line_length": 10, "m_elp_sigma": 1, "m_elp_gradient_threshold_value": 20, "m_elp_axes": [20, 100], "m_elp_ratio": [0.0, 1.0], "m_circle_inv": 1, "m_circle_type": 0, "m_circle_thr": 127, "m_circle_blur": 3, "m_circle_mean_sub": 0, "m_circle_radius": [1, 30], "m_poly_inv": 1, "m_poly_type": 0, "m_poly_thr": 127, "m_poly_blur": 3, "m_poly_mean_sub": 0, "m_poly_value": 3, "m_poly_area": [100, 100000], "m_poly_perimeter": [10, 100000], "m_cnt_inv": 1, "m_cnt_type": 0, "m_cnt_thr": 127, "m_cnt_blur": 3, "m_cnt_mean_sub": 0, "m_cnt_area": [100, 100000], "m_cnt_perimeter": [10, 100000], "m_aruco_dictionary": "DICT_6X6_250", "m_aruco_marker_length": 10, "m_aruco_refine": "CORNER_REFINE_NONE", "m_aruco_subpix": 0, "m_od_model_path":""}

        # object_detection
        self.od = Object_detection(model_path)
        
        # ocr
        self.ocr = Ocr()
        
        # thread list
        self.thread_list = []

        self.adjust_img = np.zeros((10, 10))


    def update_camera_data(self):
        depth_frame, ir_frame, color_frame, depth_img, ir_img, color_img, depth_int, frames, timestamp = self.camera.get_all()
        self.camera_data = {
            "depth_frame": depth_frame,
            "ir_frame": ir_frame,
            "color_frame": color_frame,
            "depth_img": depth_img,
            "ir_img": ir_img,
            "color_img": color_img,
            "depth_int": depth_int,
            "frames": frames,
            "timestamp": timestamp
        }
        return self.camera_data


    def img(self):
        return self.adjust_img


    def run(self, prm=None, save_path=None):        
        # camera data
        camera_data = self.update_camera_data()

        # roi

        # color

        # intensity

        # detection

        # threshould

        # pose

        # return

        # run pattern detection
        timestamp, retval, adjust_img, thr_img, bgr_img = self._pattern(self.camera, camera_data, **kwargs)
        self.adjust_img = adjust_img

        # save the plots
        if save_path:
            # Create a thread to perform the file write operation
            thread = threading.Thread(target=cv.imwrite, args=(save_path, adjust_img))
            thread.start()

            self.thread_list.append(thread)

        return retval


    """
    timestamp
    cls
    conf
    corners
    xyz
    rvec
    tvec
    """
    def _pattern(self, camera, camera_data, **kwargs):
        # init retval

        retval = []
        """bgr_img"""
        bgr_img = camera_data["color_img"]

        """adjust and out img"""
        adjust_img = bgr_img.copy()
        thr_img = bgr_img.copy()

        # intensity
        if kwargs["intensity_enb"]:
            adjust_img = intensity(adjust_img, kwargs["intensity_alpha"], kwargs["intensity_beta"])
                
        # color mask
        if kwargs["color_enb"]:
            adjust_img = color_mask(adjust_img, (kwargs["color_h"][0], kwargs["color_s"][0], kwargs["color_v"][0]), (kwargs["color_h"][1], kwargs["color_s"][1], kwargs["color_v"][1]), kwargs["color_inv"])

        # roi
        mask_img = adjust_img.copy()
        if kwargs["roi_enb"]:
            mask_img = roi_mask(mask_img, kwargs["roi_value"], kwargs["roi_inv"])

        """methods"""
        # method
        if kwargs["method_value"] == 0: # ellipse
            # edge drawing
            elps = edge_drawing(mask_img, min_path_length=kwargs["m_elp_min_path_length"], min_line_length=kwargs["m_elp_min_line_length"], nfa_validation=kwargs["m_elp_nfa_validation"], sigma=kwargs["m_elp_sigma"], gradient_threshold_value=kwargs["m_elp_gradient_threshold_value"], pf_mode=kwargs["m_elp_pf_mode"], axes=kwargs["m_elp_axes"], ratio=kwargs["m_elp_ratio"])
            
            # draw elps
            draw_ellipse(adjust_img, elps, axis=False)
            
            # corners
            corners = [get_obb_corners(elp[0], [2*elp[1][0], 2*elp[1][1]], elp[2]) for elp in elps]

            # return
            retval = [
                {"timestamp": camera_data["timestamp"],
                "cls": 0,
                "conf":1,
                "corners": corners[i],
                "xyz": [0, 0, 0],
                "rvec": [0, 0, 0],
                "tvec": [0, 0, 0],
                } for elp in elps
            ]
            for ret in retval:
                #??? center is not given
                draw_obb(adjust_img, ret["id"], ret["center"], ret["corners"])

        elif kwargs["method_value"] == 2: # polygon
            # thr
            thr_img = binary_thr(mask_img, kwargs["m_poly_type"], kwargs["m_poly_inv"], kwargs["m_poly_blur"], kwargs["m_poly_thr"], kwargs["m_poly_mean_sub"])

            # find contour
            draws = find_cnt(thr_img, kwargs["m_poly_area"], kwargs["m_poly_perimeter"], kwargs["m_poly_value"])

            # draw contours
            draw_cnt(adjust_img, draws, axis=False)

            # corners
            corners = [get_obb_corners(draw[0], draw[1], draw[2]) for draw in draws]

            # return
            retval = [
                {"timestamp": camera_data["timestamp"],
                "cls":0,
                "conf":1,
                "corners": corners[i],
                "xyz": [0, 0, 0],
                "rvec": [0, 0, 0],
                "tvec": [0, 0, 0],
                } for draw in draws
            ]
            for ret in retval:
                # ??? center is not valid
                draw_obb(adjust_img, ret["id"], ret["center"], ret["corners"])
        
        elif kwargs["method_value"] == 3: # contour
            # thr
            thr_img = binary_thr(mask_img, kwargs["m_cnt_type"], kwargs["m_cnt_inv"], kwargs["m_cnt_blur"], kwargs["m_cnt_thr"], kwargs["m_cnt_mean_sub"])

            # find contour
            draws = find_cnt(thr_img, kwargs["m_cnt_area"], kwargs["m_cnt_perimeter"])

            # draw contours
            out_img = draw_cnt(adjust_img, draws, axis=False)

            # corners
            corners = [get_obb_corners(draw[0], draw[1], draw[2]) for draw in draws]

            # return
            retval = [
                {"timestamp": camera_data["timestamp"],
                "cls": 0,
                "conf":1,
                "corners": corners[i],
                "xyz": [0, 0, 0],
                "rvec": [0, 0, 0],
                "tvec": [0, 0, 0],
                } for draw in draws
            ]
            for ret in retval:
                # ??? adjust center
                draw_obb(adjust_img, ret["id"], ret["center"], ret["corners"])
        
        elif kwargs["method_value"] == 4: # aruco #??? echck this the id does not match
            retval = []

            # pose: [id, corner, rvec, tvec]
            aruco_data = find_aruco(mask_img, camera.camera_matrix(camera_data["depth_int"]), camera.dist_coeffs(camera_data["depth_int"]), kwargs["m_aruco_dictionary"], kwargs["m_aruco_marker_length"], kwargs["m_aruco_refine"], kwargs["m_aruco_subpix"])

            for i in range(len(aruco_data)):
                _id = int(aruco_data[i][0][0])
                _timestamp = camera_data["timestamp"]

                # corner and center
                _corners = aruco_data[i][1].reshape((4,2))
                _center = [int(sum([c[0] for c in _corners])/4), int(sum([c[1] for c in _corners])/4)]
                _corners = [[int(c[0]), int(c[1])] for c in _corners]
                
                # rvec, tvec
                _rvec = aruco_data[i][2][0].tolist()
                _rvec = [r*180/np.pi for r in _rvec]
                _tvec = aruco_data[i][3][0].tolist()
                
                retval.append({
                    "id":i,
                    "center": _center,
                    "corners": _corners,
                    "xyz": [0, 0, 0],
                    "rvec": _rvec,
                    "tvec": _tvec
                    })

            # draw
            draw_aruco(adjust_img, aruco_data, camera.camera_matrix(camera_data["depth_int"]), camera.dist_coeffs(camera_data["depth_int"]))

        if kwargs["run"] == "ocr": # ocr
            retval = self.ocr.ocr(mask_img, conf=kwargs["conf"])
        if kwargs["run"] == "od": # object detection
            # run detection
            objects = self.od(mask_img, **kwargs)
            
            # format the result
            retval = [
                        {"timestamp": camera_data["timestamp"],
                        "cls": obj.cls,
                        "label": self.od["classes"][obj.cls],
                        "conf": obj.conf,
                        "corners": [[obj.rect.x, obj.rect.y], [obj.rect.x+obj.rect.w, obj.rect.y], [obj.rect.x+obj.rect.w, obj.rect.y+obj.rect.h], [obj.rect.x, obj.rect.y+obj.rect.h]],
                        "xyz": [0, 0, 0],
                        "rvec": [0, 0, 0],
                        "tvec": [0, 0, 0],
                        }
                    for obj in objects
            ]
            
            # edge drawing
            results = edge_drawing(mask_img, min_path_length=kwargs["m_elp_min_path_length"], min_line_length=kwargs["m_elp_min_line_length"], nfa_validation=kwargs["m_elp_nfa_validation"], sigma=kwargs["m_elp_sigma"], gradient_threshold_value=kwargs["m_elp_gradient_threshold_value"], pf_mode=kwargs["m_elp_pf_mode"], axes=kwargs["m_elp_axes"], ratio=kwargs["m_elp_ratio"])
            
            # draw elps
            draw_ellipse(adjust_img, elps, axis=False)
            
            # corners
            corners = [get_obb_corners(elp[0], [2*elp[1][0], 2*elp[1][1]], elp[2]) for elp in elps]

            # return
            retval = [
                {"timestamp": camera_data["timestamp"],
                "cls": 0,
                "conf":1,
                "corners": corners[i],
                "xyz": [0, 0, 0],
                "rvec": [0, 0, 0],
                "tvec": [0, 0, 0],
                } for elp in elps
            ]
            for ret in retval:
                #??? center is not given
                draw_obb(adjust_img, ret["id"], ret["center"], ret["corners"])

        # xyz:
        if camera_data["depth_frame"] is not None:
            for i in range(len(retval)):
                # xyz
                retval[i]["xyz"] = camera.xyz(retval[i]["center"], camera_data["depth_frame"], camera_data["depth_int"])[0].tolist()

        # pose
        if kwargs["method_value"] in [0, 1, 2, 3]:
            # tmp pixels from poi
            tmp_pxls = np.array(kwargs["poi_value"])
            
            if len(tmp_pxls) > 2 and camera_data["depth_frame"] is not None:
                for i in range(len(retval)):

                    # axis
                    center = retval[i]["center"]
                    dim = retval[i]["obb"]["wh"] # (w, h)
                    rot = retval[i]["obb"]["rot"]
                    del retval[i]["obb"]
                    
                    # pose from tmp
                    valid, center_3d, X, Y, Z, pxl_map = pose_3_point(camera_data["depth_frame"], camera_data["depth_int"], tmp_pxls, center, dim, rot, camera)

                    if valid: # add pose
                        draw_3d_axis(adjust_img, center_3d, X, Y, Z, camera.camera_matrix(camera_data["depth_int"]), camera.dist_coeffs(camera_data["depth_int"]))
                        
                        retval[i]["tvec"] = center_3d.tolist()
                        rodrigues, _= cv.Rodrigues(np.matrix([[X[0], Y[0], Z[0]],
                                                [X[1], Y[1], Z[1]],
                                                [X[2], Y[2], Z[2]]])) 
                        retval[i]["rvec"] = [rodrigues[i, 0]*180/np.pi for i in range(3)]
                    

                    # draw template
                    for pxl in pxl_map:
                        draw_point(adjust_img, pxl)

        return camera_data["timestamp"], retval, adjust_img, thr_img, bgr_img

########################################################
########################################################








class default_widget(object):
    """docstring for ClassName"""
    def __init__(self,):
        super(default_widget, self).__init__()
        continuous_update = False
        style={'description_width': '150px'}
        self.widget_input = {
            "poi_label": widgets.Label(value="Select 3 points relative to the oriented bounding box to define a hyperplane and determine the 6D pose of the detected item.", layout={'width': '99%'}, style=style),
            "poi_enb": widgets.Checkbox(value=False, description='Apply the 6D-pose', continuous_update=continuous_update, layout={'width': '99%'}, style=style),
            "poi_value": widgets.Text(value="[]", placeholder='[]', description='POI', disabled=True, layout={'width': '99%'}, style=style),
            
            "color_label": widgets.Label(value="Apply a color mask to filter specific colors by adjusting hue, saturation, and value. Fine-tune these settings to isolate the desired color range for better detection results.", layout={'width': '99%'}, style=style),
            "color_enb": widgets.Checkbox(value=False, description='Apply the color mask', continuous_update=continuous_update, layout={'width': '99%'}, style=style),
            "color_h": widgets.IntRangeSlider(value=[60, 120], min=0, max=179, step=1, description='Hue', continuous_update=continuous_update, layout={'width': '99%'}, style=style),
            "color_s": widgets.IntRangeSlider(value=[85, 170], min=0, max=255, step=1, description='Saturation', continuous_update=continuous_update, layout={'width': '99%'}, style=style),
            "color_v": widgets.IntRangeSlider(value=[85, 170], min=0, max=255, step=1, description='Vue', continuous_update=continuous_update, layout={'width': '99%'}, style=style),
            "color_inv": widgets.Checkbox(value=False, description='Invert the mask', continuous_update=continuous_update, layout={'width': '99%'}, style=style),

            "roi_label": widgets.Label(value="Select the region of interest where the detection method is applied. Use the blue polygon selector on the output image to define this area.", layout={'width': '99%'}, style=style),
            "roi_enb": widgets.Checkbox(value=False, description='Apply the ROI', continuous_update=continuous_update, layout={'width': '99%'}, style=style),
            "roi_value": widgets.Text(value='[]', placeholder='[]', description='ROI', disabled=True, layout={'width': '99%'}, style=style),
            "roi_inv": widgets.Checkbox(value=False, description='Invert the region', continuous_update=continuous_update, layout={'width': '99%'}, style=style),

            "intensity_label": widgets.Label(value="Adjust brightness and contrast if necessary to enhance image details. Use the sliders for optimal visibility and improved detection results.", layout={'width': '99%'}, style=style),
            "intensity_enb": widgets.Checkbox(value=False, description='Apply the intensity', continuous_update=continuous_update, layout={'width': '99%'}, style=style),
            "intensity_alpha" : widgets.FloatSlider(value=2, min=0, max=4, step=0.01, description='Contrast', continuous_update=continuous_update, layout={'width': '99%'}, style=style),
            "intensity_beta" : widgets.IntSlider(value=50, min=0, max=255, step=1, description='Brightness', continuous_update=continuous_update, layout={'width': '99%'}, style=style),

            "2d_range_label": widgets.Label(value="Set constraints on the detected item's oriented bounding box aspect ratio and largest dimension to refine detection accuracy.", layout={'width': '99%'}, style=style),
            "2d_range_enb": widgets.Checkbox(value=False, description='Apply the size constraints', continuous_update=continuous_update,layout={'width': '99%'}, style=style),
            "2d_range_aspect_ratio": widgets.FloatRangeSlider(value=[0, 1], min=0, max=1, step=0.01, description='Aspect ratio', continuous_update=continuous_update, layout={'width': '99%'}, style=style),
            "2d_range_size_range": widgets.IntRangeSlider(value=[0, 1280], min=0, max=1280, step=10, description='Largest dimension (px)', continuous_update=continuous_update, layout={'width': '99%'}, style=style),

            "3d_range_label": widgets.Label(value="Apply spatial constraints to remove detected items outside the specified x, y, z range relative to the frame.", layout={'width': '99%'}, style=style),
            "3d_range_enb": widgets.Checkbox(value=False, description='Apply the spatial constraints', continuous_update=continuous_update, layout={'width': '99%'}, style=style),
            "3d_range_x": widgets.IntRangeSlider(value=[250, 350], min=-750, max=750, step=1, description='x (mm)', continuous_update=continuous_update, layout={'width': '99%'}, style=style),
            "3d_range_y": widgets.IntRangeSlider(value=[0, 50], min=-750, max=750, step=1, description='y (mm)', continuous_update=continuous_update, layout={'width': '99%'}, style=style),
            "3d_range_z": widgets.IntRangeSlider(value=[0, 50], min=-750, max=750, step=1, description='z (mm)', continuous_update=continuous_update, layout={'width': '99%'}, style=style),
            "3d_range_inv": widgets.Checkbox(value=False, description='Invert the range', continuous_update=continuous_update, layout={'width': '99%'}, style=style),

            "method_value": widgets.Dropdown(value=0, options=[('Ellipse detection', 0), ('Polygon detection', 2), ('Contour detection', 3), ('Aruco detection', 4), ('OCR detection', 5), ('Object detection', 6),], description='Detection method', continuous_update=continuous_update, style=style),
    
            "m_elp_pf_mode": widgets.Checkbox(value=False, description='Auto detection', continuous_update=continuous_update,layout={'width': '99%'}, style=style),
            "m_elp_nfa_validation": widgets.Checkbox(value=True, description='False alarm validation', continuous_update=continuous_update, layout={'width': '99%'}, style=style),
            "m_elp_min_path_length": widgets.IntSlider(value=50, min=1, max=1000, step=1, description='Min path length', continuous_update=continuous_update, layout={'width': '99%'}, style=style),
            "m_elp_min_line_length": widgets.IntSlider(value=10, min=1, max=1000, step=1, description='Min line length', continuous_update=continuous_update, layout={'width': '99%'}, style=style),
            "m_elp_sigma": widgets.IntSlider(value=1, min=0, max=20, step=0.1, description='Blur', continuous_update=continuous_update, layout={'width': '99%'}, style=style),
            "m_elp_gradient_threshold_value": widgets.IntSlider(value=20, min=1, max=100, step=1, description='Gradient', continuous_update=continuous_update, layout={'width': '99%'}, style=style),
    

            "m_poly_inv": widgets.Checkbox(value=True, description='Inverse', continuous_update=continuous_update, layout={'width': '99%'}, style=style),
            "m_poly_type": widgets.Dropdown(value=0, options=[('0: Otsu (auto)', 0), ('1: Binary', 1), ('2: Gaussian', 2)], description='Type', continuous_update=continuous_update, layout={'width': '99%'}, style=style),
            "m_poly_thr" : widgets.IntSlider(value=127, min=0, max=255, step=1, description='Threshold value', continuous_update=continuous_update, layout={'width': '99%'}, style=style),
            "m_poly_blur": widgets.IntSlider(value=3, min=1, max=20, step=1, description='Smoothing blur', continuous_update=continuous_update, layout={'width': '99%'}, style=style),                    
            "m_poly_mean_sub": widgets.IntSlider(value=0, min=-200, max=200, step=1, description='Mean subtract', continuous_update=continuous_update, layout={'width': '99%'}, style=style),
            "m_poly_value" : widgets.IntSlider(value=3, min=3, max=20, step=1, description='Sides', continuous_update=continuous_update, layout={'width': '99%'}, style=style),

            
            "m_cnt_inv": widgets.Checkbox(value=True, description='Inverse', continuous_update=continuous_update, layout={'width': '99%'}, style=style),
            "m_cnt_type": widgets.Dropdown(value=0, options=[('0: Otsu (auto)', 0), ('1: Binary', 1), ('2: Gaussian', 2)], description='Type', continuous_update=continuous_update, layout={'width': '99%'}, style=style),
            "m_cnt_thr" : widgets.IntSlider(value=127, min=0, max=255, step=1, description='Threshold value', continuous_update=continuous_update, layout={'width': '99%'}, style=style),
            "m_cnt_blur": widgets.IntSlider(value=3, min=1, max=20, step=1, description='Smoothing blur', continuous_update=continuous_update, layout={'width': '99%'}, style=style),                    
            "m_cnt_mean_sub": widgets.IntSlider(value=0, min=-200, max=200, step=1, description='Mean subtract', continuous_update=continuous_update, layout={'width': '99%'}, style=style),

            "m_aruco_dictionary":widgets.Dropdown(value="DICT_6X6_250", options= ["DICT_4X4_50", "DICT_4X4_100", "DICT_4X4_250", "DICT_4X4_1000", "DICT_5X5_50", "DICT_5X5_100", "DICT_5X5_250", "DICT_5X5_1000", "DICT_6X6_50", "DICT_6X6_100", "DICT_6X6_250", "DICT_6X6_1000", "DICT_7X7_50", "DICT_7X7_100", "DICT_7X7_250", "DICT_7X7_1000", "DICT_ARUCO_ORIGINAL", "DICT_APRILTAG_16h5", "DICT_APRILTAG_25h9", "DICT_APRILTAG_36h10", "DICT_APRILTAG_36h11"], description='Dictionary', continuous_update=continuous_update, layout={'width': '99%'}, style=style),
            "m_aruco_marker_length": widgets.IntSlider(value=10, min=1, max=100, step=1, description='Marker length (mm)', continuous_update=continuous_update, layout={'width': '99%'}, style=style),
            "m_aruco_refine":widgets.Dropdown(value="CORNER_REFINE_APRILTAG", options=["CORNER_REFINE_NONE", "CORNER_REFINE_SUBPIX", "CORNER_REFINE_CONTOUR", "CORNER_REFINE_APRILTAG"], description='Refinement', continuous_update=continuous_update, layout={'width': '99%'}, style=style),
            "m_aruco_subpix": widgets.Checkbox(value=False, description='Sub pixel', continuous_update=continuous_update, layout={'width': '99%'}, style=style),            

            "m_ocr_crop": widgets.Checkbox(value=False, description='Crop ROI', continuous_update=continuous_update, layout={'width': '99%'}, style=style),
            "m_ocr_conf" : widgets.FloatSlider(value=0.5, min=0.01, max=0.99, step=0.01, description='Confidence', continuous_update=continuous_update, layout={'width': '99%'}, style=style), 

            "m_od_crop": widgets.Checkbox(value=False, description='Crop ROI', continuous_update=continuous_update, layout={'width': '99%'}, style=style),
            "m_od_conf" : widgets.FloatSlider(value=0.5, min=0.01, max=0.99, step=0.01, description='Confidence', continuous_update=continuous_update, layout={'width': '99%'}, style=style), 
            "m_od_max_det" : widgets.IntSlider(value=10, min=1, max=100, step=1, description='Max detection', continuous_update=continuous_update, layout={'width': '99%'}, style=style), 
            "m_od_cls" : widgets.Text(value="[]", placeholder='[]', description='Detection classes', disabled=False, layout={'width': '99%'}, style=style), 

        }
        self.widget_trigger = {
            "color_picker": widgets.ColorPicker(concise=False, description='Color picker', value='blue', disabled=False, style={'text_width': '0'}),
            "color_hsv": widgets.Text(value='Hue = 119, Saturation = 255, Value = 255', placeholder='', description='', disabled=True, layout={'width': '99%'}),            

            "source_value": widgets.Dropdown(value=0, options=[('Stereo camera', 0)], description='Camera', continuous_update=continuous_update, layout={'layout': '99%'}),
            "s_file_value": widgets.Text(value='', placeholder='Path to the file (*.jpg, *.jpeg, *.png, *.tiff, ...).Ex: test.jpg', description='File path', disabled=False, layout={'width': '99%'}),            
            "s_apply": widgets.Button( description='Capture Image', disabled=False, button_style="", tooltip='Capture Image'),
            "s_update": widgets.Button( description='', disabled=False, button_style="", tooltip='Update source list', icon='refresh', layout={'width': '50px'}),
            "s_save_path": widgets.Text(value='', placeholder='*.jpg', description='Save image as', disabled=False, layout={'width': '99%'}),            
            "s_save": widgets.Button( description='Save', disabled=False, button_style="", tooltip='Save as'),

            #"model_path": widgets.Text(value='', placeholder='/full_path/to_the/object_detection_model.pkl', description='Object Detection Model', disabled=False, layout={'width': '99%'}, style=style),            
            #"model_save": widgets.Button( description='Set', disabled=False, button_style="", tooltip='Set'),

            #"robot_ip": widgets.Text(value='', placeholder='192.168.254.10', description='Robot IP Address', disabled=False, layout={'width': '99%'}, style=style),            
            #"robot_connect": widgets.Button( description='Connect', disabled=False, button_style="", tooltip='Connect'),

            #"camera_robot_calibration": widgets.Textarea(value='[[0.00525873615, -0.999894519, 0.0134620306, 46.5174596], [0.999959617, 0.00535678348, -0.00735796480, 32.0776662], [0.00728773209, -0.0135001806, 0.999882310, -4.24772615], [0.0, 0.0, 0.0, 1.0]]', placeholder='[[0.00525873615, -0.999894519, 0.0134620306, 46.5174596], [0.999959617, 0.00535678348, -0.00735796480, 32.0776662], [0.00728773209, -0.0135001806, 0.999882310, -4.24772615], [0.0, 0.0, 0.0, 1.0]]', description='Camera & Robot Calibration Matrix', disabled=False, layout={'width': '99%'}, style=style),            

            "out_prm": widgets.Textarea(value='', placeholder='', description='Configuration', disabled=True, layout={'width': '99%'}),
            "out_return": widgets.Textarea(value='', placeholder='', description='Return value', disabled=True, layout={'width': '99%'}),

            "close": widgets.Button( description='Terminate App', disabled=False, button_style="danger", tooltip='Terminate App'),

        }

class App(object):
    """docstring for App"""
    def __init__(self):
        super(App, self).__init__()
        self.retval ={}
        self.config = {}

    def close(self, b):
        self.d.close()
        self.d.camera.close()
        plt.close('all')

    def run(self, ip_address, model_path=None, calibration_matrix=None, frame=None):        
        # camera
        camera_connect = False
        camera = Camera()
        try:
            camera.connect()
            camera_connect = True
            self.camera_connect = camera_connect
        except Exception as ex:
            print(ex)
            pass
        
        # detect
        self.d = Detection(camera)
        
        # capture the first 
        if camera_connect:
            self.d.update_camera_data()
        else:
            self.d.camera_data = {"depth_frame": None, "ir_frame": None, "color_frame": None, "depth_img": None, "ir_img": None, "color_img": np.zeros((10, 10, 3), dtype=np.uint8), "depth_int": None, "frames": None, "timestamp": 0}
    
        """out plot"""
        # close everything first
        plt.close('all')
        
        # Create an initial display with the original image
        fig, ax = plt.subplots(frameon=False)
        self.plt = {
            "out":{
                "fig": fig,
                "ax": ax,
                "img_plt": ax.imshow(cv.cvtColor(self.d.camera_data["color_img"], cv.COLOR_BGR2RGB))
        }}
        #self.plt["out"]["fig"].suptitle("Output image")
        self.plt["out"]["fig"].canvas.header_visible = False
        self.plt["out"]["fig"].tight_layout()

        
        """widgets"""
        # widget
        self.widget_in = default_widget().widget_input
        self.widget_tr = default_widget().widget_trigger
        
        """accordion for adjust the image"""
        # adjust_image
        color_picker_box = widgets.HBox([self.widget_tr[k] for k in [key for key in self.widget_tr.keys() if key.startswith('color_')]])
        acc_adjust_img = widgets.Accordion()
        acc_adjust_img.children = [
            widgets.VBox([self.widget_in[k] for k in [key for key in self.widget_in.keys() if key.startswith('roi_')]]),
            widgets.VBox([self.widget_in[k] for k in [key for key in self.widget_in.keys() if key.startswith('intensity_')]]),
            widgets.VBox([self.widget_in[k] for k in [key for key in self.widget_in.keys() if key.startswith('color_')]]+[widgets.HTML("<hr>")]+[color_picker_box]),
        ]

        for i, title in enumerate(['Region of Interest', 'Intensity', 'Color Mask']):
            acc_adjust_img.set_title(i, title)    

        """source vbox"""
        source_vbox = widgets.VBox([
            widgets.HBox([self.widget_tr[k] for k in [key for key in ["source_value", "s_file_value", "s_apply"]]]),
            widgets.VBox([widgets.HTML("<hr>")]),
            #widgets.HBox([self.widget_tr[k] for k in [key for key in ["model_path", "model_save"]]]),
            #widgets.VBox([widgets.HTML("<hr>")]),
            #widgets.HBox([self.widget_tr[k] for k in [key for key in ["robot_ip", "robot_connect"]]]),
            #widgets.HBox([self.widget_tr[k] for k in [key for key in ["s_save_path","s_save"]]]),
            #widgets.VBox([widgets.HTML("<hr>")]),
            #widgets.HBox([self.widget_tr[k] for k in [key for key in ["camera_robot_calibration"]]]),
            #widgets.VBox([widgets.HTML("<hr>")]),
            widgets.HBox([self.widget_tr[k] for k in [key for key in ["close"]]]),
        ])

        """method vbox"""
        method_vbox = widgets.VBox([
            self.widget_in["method_value"],
            widgets.VBox([widgets.HTML("<hr>")]),
            widgets.VBox([self.widget_in[k] for k in [key for key in self.widget_in.keys() if key.startswith('m_elp_')]]),
            widgets.VBox([self.widget_in[k] for k in [key for key in self.widget_in.keys() if key.startswith('m_circle_')]]),
            widgets.VBox([self.widget_in[k] for k in [key for key in self.widget_in.keys() if key.startswith('m_poly_')]]),
            widgets.VBox([self.widget_in[k] for k in [key for key in self.widget_in.keys() if key.startswith('m_cnt_')]]),
            widgets.VBox([self.widget_in[k] for k in [key for key in self.widget_in.keys() if key.startswith('m_aruco_')]]),
            widgets.VBox([self.widget_in[k] for k in [key for key in self.widget_in.keys() if key.startswith('m_ocr_')]]),
            widgets.VBox([self.widget_in[k] for k in [key for key in self.widget_in.keys() if key.startswith('m_od_')]]),
        ], layout={'width': '100%'})        

        """POI"""
        # poi selector
        poi_plot = widgets.Output()
        with poi_plot:
            # init fig and ax
            fig, ax = self.poi_plt()
            self.plt["poi"] = {
                "fig": fig,
                "ax": ax
            }
            plt.show(self.plt["poi"]["fig"])  

        # init poi
        self.poi_value = poly_select(self.widget_in["poi_value"])

        # Initialize poi selector
        self.poi_selector = PolygonSelector(self.plt["poi"]["ax"], onselect=self.poi_value.onselect, useblit=True, props=dict(color='orange', linestyle='--'))
        poi_box = widgets.VBox([self.widget_in["poi_value"], poi_plot])


        """accordion for spatial constraints"""
        # spatial constraints
        acc_spatial = widgets.Accordion()
        acc_spatial.children = [
            widgets.VBox([self.widget_in[k] for k in [key for key in self.widget_in.keys() if key.startswith('2d_range_')]]),
            widgets.VBox([self.widget_in[k] for k in [key for key in self.widget_in.keys() if key.startswith('3d_range_')]]),
            poi_box,
        ]

        for i, title in enumerate(["Size ", 'Location', '6D-Pose']):
            acc_spatial.set_title(i, title)    

        """method plot"""
        method_plt = widgets.Output(layout={'width': '100%'})
        with method_plt:
            # init fig and ax
            fig, ax = plt.subplots(frameon=False)
            self.plt["method"] = {
                "fig": fig,
                "ax": ax,
                "img_plt": ax.imshow(self.d.camera_data["color_img"], cmap='gray')
            }
            self.plt["method"]["fig"].canvas.header_visible = False
            self.plt["method"]["fig"].tight_layout()
            self.plt["method"]["fig"].set_size_inches((4,3), forward=True)
            self.plt["method"]["img_plt"].set_visible(False)
            self.plt["out"]["ax"].axis('off')
            plt.show(self.plt["method"]["fig"])

        """result"""
        result_vbox = widgets.VBox([self.widget_tr[k] for k in [key for key in self.widget_tr.keys() if key.startswith('out_')]])

        """tab"""
        tab = widgets.Tab()
        tab.children = [
            source_vbox,
            acc_adjust_img,
            widgets.HBox([method_vbox, method_plt]),
            acc_spatial,
            result_vbox,
        ]
        tab.set_title(0, 'Initialization')
        tab.set_title(1, 'Visual Adjustment')
        tab.set_title(2, 'Detection')
        tab.set_title(3, 'OBB')
        tab.set_title(4, 'Result')
        display(tab)

        """roi"""
        # init roi
        self.roi_value = poly_select(self.widget_in["roi_value"])

        # Initialize PolygonSelector
        self.roi_selector = PolygonSelector(self.plt["out"]["ax"], onselect=self.roi_value.onselect, useblit=True, props=dict(color='blue', linestyle='--'))

        # interactive for source
        interactive(self.hide_show_source, source_value=self.widget_tr["source_value"])

        # interactive color_picker
        self.widget_tr["color_picker"].observe(self.hex_to_hsv, names='value')
        
        # capture
        self.widget_tr["s_apply"].on_click(self.capture_camera_data)

        # capture
        self.widget_tr["s_update"].on_click(self.update_source_list)

        # save
        self.widget_tr["s_save"].on_click(self.save_as_source)

        # close
        self.widget_tr["close"].on_click(self.close)

        # Create an interactive plot with the slider
        interactive(self._detect_pattern, **self.widget_in)
        
        
    def hex_to_hsv(self, change):
        # Remove '#' if present
        hex_color = change['new'].lstrip('#')

        # Convert hex to RGB
        rgb_color = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

        # Normalize RGB values to the range [0, 1]
        normalized_rgb = tuple(value / 255.0 for value in rgb_color)

        # Convert RGB to HSV
        hsv_color = colorsys.rgb_to_hsv(*normalized_rgb)

        # Adjust HSV values to the common OpenCV conventions
        h = int(hsv_color[0] * 179)
        s = int(hsv_color[1] * 255)
        v = int(hsv_color[2] * 255)
        self.widget_tr["color_hsv"].value = f"Hue = {h}, Saturation = {s}, Value = {v}"

        # color
        self.widget_in["color_h"].value = [max(0, h-20), min(179, h+20)]
        self.widget_in["color_s"].value = [max(0, s-20), min(255, s+20)]
        self.widget_in["color_v"].value = [max(0, v-20), min(255, v+20)]


    
    def save_as_source(self, b):
        file_path = self.widget_tr["s_save_path"].value
        
        # opencv
        cv.imwrite(file_path, self.d.camera_data["color_img"])

        
    def open_pkl(self, file_path):
        with open(file_path, 'rb') as file:
            loaded_data = pkl.load(file)
        return loaded_data


    def capture_camera_data(self, b):
        if self.widget_tr["source_value"].value == 1: # read from file
            # file_path
            file_path = self.widget_tr["s_file_value"].value
            
            if file_path.endswith(".pkl"): # picle format
                # data
                self.d.camera_data = self.open_pkl(file_path)
            else:
                img = cv.imread(file_path)
                self.d.camera_data = {"depth_frame": None, "ir_frame": None, "color_frame": None, "depth_img": None, "ir_img": None, "color_img": img, "depth_int": None, "frames": None, "timestamp": 0}

        elif self.widget_tr["source_value"].value == 0: # d405
            # get the data from camera
            self.d.update_camera_data()

        # adjust the frame size
        self.plt["out"]["img_plt"].set_extent([0, self.d.camera_data["color_img"].shape[1], self.d.camera_data["color_img"].shape[0], 0])
        self.plt["method"]["img_plt"].set_extent([0, self.d.camera_data["color_img"].shape[1], self.d.camera_data["color_img"].shape[0], 0])

        # call update
        kwargs = {k:self.widget_in[k].value for k in self.widget_in.keys()}
        self._detect_pattern(**kwargs)     

    def update_source_list(self, b):
        all_device = self.d.camera.all_device()
        
        i = 0
        options = []
        for device in all_device:
            options.append((device["name"] +" (S/N: "+device["serial_number"], ")", i))
            i += 1
        options.append(('Image file', i))
        self.widget_tr["source_value"].options = options
        
    def hide_show_source(self, **kwargs):
        if kwargs["source_value"] == 1:
            self.widget_tr["s_file_value"].layout.display = "flex"
        elif kwargs["source_value"] == 0:
            self.widget_tr["s_file_value"].layout.display = "none"


    def poi_plt(self):
        # create
        fig, ax = plt.subplots(frameon=False)
        #fig.suptitle("Select 3 points of interest")
        fig.canvas.header_visible = False
        fig.tight_layout()
        
        # Set the height and calculate the width based on the golden ratio
        height = 1.0
        width = 1.0

        # Draw the ellipse in magenta
        ellipse = Ellipse((0, 0), 2 * width, 2 * height, linewidth=1, edgecolor='#FF00FF', facecolor='none')
        ax.add_patch(ellipse)

        # Draw the minimum bounding box around the ellipse in magenta
        min_bounding_box = plt.Rectangle((-width, -height), 2 * width, 2 * height, linewidth=1, edgecolor='#FF00FF', facecolor='none', label='Oriented Bounding Box')
        ax.add_patch(min_bounding_box)

        # Draw major and minor axes
        major_axis = plt.Line2D([0, width], [0, 0], color='red', linestyle='dashed', linewidth=1, label='Major Axis')
        minor_axis = plt.Line2D([0, 0], [0, height], color='green', linestyle='dashed', linewidth=1, label='Minor Axis')
        ax.add_line(major_axis)
        ax.add_line(minor_axis)

        # Plot the center of the rectangle in blue
        ax.plot(0, 0, marker='o', markersize=6, color='blue', label='Center')

        # Set axis limits with x and y axes twice as large
        ax.set_xlim(-2 * width, 2 * width)
        ax.set_ylim(-2 * height, 2 * height)

        # Display the legend
        ax.legend()

        # Set aspect ratio
        ax.set_aspect(1/1.68)

        # invert y
        ax.invert_yaxis()

        # Display the plot
        return fig, ax

    
    def _detect_pattern(self, **kwargs):
        try:
            # adjust kwargs
            kwargs["roi_value"] = ast.literal_eval(kwargs["roi_value"])
            kwargs["poi_value"] = ast.literal_eval(kwargs["poi_value"])

            # adjust kwargs
            prm = {}
            """
            #roi
            if kwargs["roi_enb"]:
                prm.append()
            """
            # run pattern detection
            timestamp, retval, adjust_img, thr_img, _ = self.d._pattern(self.d.camera, self.d.camera_data, **kwargs)


            """hide and show inputs"""
            show_key = [[key for key in self.widget_in.keys() if key.startswith(term)] for term in ["m_elp", "m_circle", "m_poly", "m_cnt", "m_aruco", "m_ocr", "m_od"]][kwargs["method_value"]]
            hide_key = [key for key in self.widget_in.keys() if key.startswith('m_') and key not in show_key] 
            self.xxx = show_key
            for k in show_key:
                if self.widget_in[k].layout.display != "flex":
                    self.widget_in[k].layout.display = "flex"
            for k in hide_key:
                if self.widget_in[k].layout.display != "none":
                    self.widget_in[k].layout.display = "none"

            # display thr
            if kwargs["method_value"] in [0, 4, 5, 6]: # ellipse, aruco      
                # img
                self.plt["method"]["img_plt"].set_visible(False)
                self.plt["method"]["ax"].axis('off')


            elif kwargs["method_value"] in [1, 2, 3]: # polygon and contour
                # img
                self.plt["method"]["img_plt"].set_visible(True)
                self.plt["method"]["ax"].axis('on')
                self.plt["method"]["img_plt"].set_data(thr_img)


            # Update the existing plot
            self.plt["out"]["img_plt"].set_data(cv.cvtColor(adjust_img, cv.COLOR_BGR2RGB))

            # Redraw the plot
            self.plt["out"]["fig"].canvas.draw_idle()
            self.plt["method"]["fig"].canvas.draw_idle()

            # type retval
            json_str = json.dumps(retval)
            converted_retval = json.loads(json_str, parse_int=lambda x: int(x), parse_float=lambda x: float(x), parse_constant=lambda x: x, object_hook=lambda d: {k: 1 if v is True else 0 if v is False else v for k, v in d.items()}) 
            self.widget_tr["out_return"].value = json.dumps(converted_retval)
            self.retval = retval

            # parameters
            json_str = json.dumps(kwargs)
            converted_kwargs = json.loads(json_str, parse_int=lambda x: int(x), parse_float=lambda x: float(x), parse_constant=lambda x: x, object_hook=lambda d: {k: 1 if v is True else 0 if v is False else v for k, v in d.items()}) 
            self.widget_tr["out_prm"].value = json.dumps(converted_kwargs)
            self.config = kwargs
        except Exception as ex:
            print(ex)
            pass

In [ ]:
robot_ip_address = ""
object_detection_model_path = ""
camera_robot_calibration_matrix = None
frame = None
x = App()
x.run(robot_ip_address, object_detection_model_path, camera_robot_calibration_matrix, frame)